## Figure 6: Climate projections (GCM and SSP)

In [ ]:
# spatial libraries
import shapely.geometry
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe
import regionmask
from xclim.indicators import atmos
from xclim import core 

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import cftime
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# colors
cl = px.colors.qualitative.D3
cs = px.colors.sequential.Sunset

os.chdir('/home/rooda/Dropbox/Patagonia/')
period = slice("1980-01-01", "2019-12-31")

In [ ]:
# Catchment shapefiles
basins = gpd.read_file("GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

names = ["Yelcho", "Baker", "Santa Cruz                           ", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"]
basins.loc[basins.basin_area > 5000, "Name"] = names

# Glacier shapefiles
rgi6 = gpd.read_file("GIS South/Glaciers/RGI6_v2.shp")[["geometry"]]
rgi7 = gpd.read_file("GIS South/Glaciers/RGI7_v2.shp")[["geometry"]]
glaciers  = pd.concat([rgi6.geometry, rgi7.geometry])
glaciers  = glaciers.buffer(0.05) # mask to use for baseline climate

# high resolution grid
raster_hr   = xr.open_dataset("/home/rooda/OGGM_results/PMET_OGGM_1980_2019m.nc").prcp

## Raw data

In [ ]:
os.chdir("/home/rooda/OGGM_results/Future_climate/")

lat_coords = np.arange(-56,-40, 0.5)
lon_coords = np.arange(-76,-67, 0.5)

baseline_period = slice("1980-01-01", "2010-01-01")
future_period   = slice("2070-01-01", "2100-01-01")

gcm_list  = ["ACCESS-CM2", "BCC-CSM2-MR", "CMCC-ESM2", "FGOALS-f3-L", "GFDL-ESM4", "CMCC-CM2-SR5", "KACE-1-0-G", "MPI-ESM1-2-HR", "MRI-ESM2-0", "MIROC6"]
ssp_list  = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

results_pp = []
results_t2m = []

for ssp in tqdm(ssp_list):
    
    results_gcm_pp  = []
    results_gcm_t2m = []
    
    for gcm in gcm_list:
        
        pp_model_ssp = xr.open_dataset("PP_" + gcm + "_" + ssp + ".nc")["pr"]
        pp_model_ssp = pp_model_ssp.interp(lat = lat_coords, lon = lon_coords)
        pp_model_ssp = core.units.convert_units_to(pp_model_ssp, target = 'mm month-1', context = "hydro").resample(time = "YS").sum()
        pp_change    = (pp_model_ssp.sel(time = future_period).mean(dim="time") / pp_model_ssp.sel(time = baseline_period).mean(dim="time"))-1
        results_gcm_pp.append(pp_change)
        
        t2m_model_ssp = xr.open_dataset("T2M_" + gcm + "_" + ssp + ".nc")["tas"]
        t2m_model_ssp = t2m_model_ssp.interp(lat = lat_coords, lon = lon_coords)
        t2m_model_ssp = t2m_model_ssp.resample(time='YS').mean()        
        t2m_change    = t2m_model_ssp.sel(time = future_period).mean(dim="time") - t2m_model_ssp.sel(time = baseline_period).mean(dim="time")
        results_gcm_t2m.append(t2m_change)
        
    results_gcm_pp  = xr.concat(results_gcm_pp,  dim='gcm')
    results_gcm_t2m = xr.concat(results_gcm_t2m, dim='gcm')
    results_pp.append(results_gcm_pp)
    results_t2m.append(results_gcm_t2m)
    
dataset = xr.merge([xr.concat(results_pp,  dim='ssp'), 
                    xr.concat(results_t2m, dim='ssp')])

In [ ]:
# GCM uncertainty (SSP 245) > 80% of the models should agree the direction
gcm_spread = dataset.pr[2].where(dataset.pr[1] >= 0, 1).where(dataset.pr[2] < 0, -1).sum(dim = "gcm")
gcm_spread = gcm_spread.to_dataframe().reset_index()

gcm_spread = gpd.GeoDataFrame(gcm_spread.pr, geometry=gpd.points_from_xy(gcm_spread.lon,gcm_spread.lat))
gcm_spread = gcm_spread[gcm_spread.pr.abs() < 8]

In [ ]:
## resample using ESMF.RegridMethod.NEAREST_STOD
regridder  = xe.Regridder(dataset,  raster_hr, "bilinear")
dataset    = regridder(dataset)

# only glacier area
mask    = regionmask.mask_geopandas(glaciers, dataset)   >= 0
dataset = dataset.where(mask, drop = True)

# multi-model mean
dataset = dataset.mean(dim = "gcm")

In [ ]:
# select SSP 245 for map (a,b)
savg  = xe.SpatialAverager(dataset,  basins.geometry, geom_dim_name="avg")

basins["PP_change_126"] = savg(dataset.pr[0], skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values*100
basins["PP_change_245"] = savg(dataset.pr[1], skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values*100
basins["PP_change_370"] = savg(dataset.pr[2], skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values*100
basins["PP_change_585"] = savg(dataset.pr[3], skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values*100

basins["T2M_change_126"] = savg(dataset.tas[0], skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_change_245"] = savg(dataset.tas[1], skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_change_370"] = savg(dataset.tas[2], skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_change_585"] = savg(dataset.tas[3], skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

# major catchments
basins_m = basins.dropna(subset = ['Name'])

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)
geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords  = [-43.2, -45.95,  -46.9,  -47.55,   -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords  = [-71.2, -71.7,   -73.8,   -71.7,   -72.2,   -72.3,   -72.1, -72.8, -69.2]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

In [ ]:
fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.03, column_widths = [0.34, 0.34, 0.32], shared_xaxes = True,
                    subplot_titles = ["Precipitation change (ΔPP)","Temperature change (ΔT2M)","Changes by scenario"],
                    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "scattergeo", "rowspan": 2}, {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}]])

## Basemap
for x in range(1,3):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["rgba(213,213,213,0.6)", "rgba(213,213,213,0.6)"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x)

# Precipitation mean change (a) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_change_245'], 
                            colorscale = [(0.,"#fe7e0d"),(0.75, "#ffe9ba"),(1, "#1d78b4")], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = -15, zmax = 5, colorbar=dict(len=0.45, x=0.24, y= 0.75, title='ΔPP (%)', thickness=20)), row=1, col=1)

fig.add_trace(go.Scattergeo(lat=gcm_spread.geometry.y, lon=gcm_spread.geometry.x, mode="markers", showlegend=False, 
                            marker=dict(color="black", size=3, opacity=0.2)), row=1, col=1)
fig.add_annotation(text="a)", font=dict(size=16), x=0.005, y=0.995,  xref = "paper", yref = "paper", showarrow=False)

# Temperature mean change (b) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_change_245'], 
                            colorscale=[cs[0], cs[3],  cs[5]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = 1.2, zmax = 1.8, colorbar=dict(len=0.45, x=0.58, y= 0.75, title='ΔT2M (ºC)', dtick = 0.2, thickness=20)), row=1, col=2)
fig.add_annotation(text="b)", font=dict(size=16), x=0.36, y=0.995,  xref = "paper", yref = "paper", showarrow=False)

# layout a) and b)
for x in range(1,3):
    ## Add basin and hydrological zone names plus the hydro zone divides
    fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7,color = 'black'),opacity = 0.5, showlegend = False),row=1, col=x)  
    fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.5)"), showlegend = False),row=1, col=x)
    fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False,
                       textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=x)

fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "rgb(255,255,255)", showland = False, showcoastlines = False, showlakes = False)

# Precipitation ssp spread (c) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = basins["PP_change_126"], marker_color= cl[2], name = "SSP126", points=False, opacity=0.7, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins["PP_change_245"], marker_color= cl[0], name = "SSP245", points=False, opacity=0.7, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins["PP_change_370"], marker_color= cl[1], name = "SSP370", points=False, opacity=0.7, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins["PP_change_585"], marker_color= cs[5], name = "SSP585", points=False, opacity=0.7, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins_m["PP_change_126"],  marker_color=cl[2], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "SSP126",  showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins_m["PP_change_245"],  marker_color=cl[0], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "SSP245",  showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins_m["PP_change_370"],  marker_color=cl[1], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "SSP370",  showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins_m["PP_change_585"],  marker_color=cs[5], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "SSP585",  showlegend = False), row=1, col=3)
fig.update_traces(pointpos = -0.7, box_visible=True, meanline_visible=True, width=1, row = 1, col = 3)

fig.update_yaxes(title = "Precipitation change (%)", title_standoff = 0, range = [-25,10], side = "right", row = 1, col = 3)
fig.add_annotation(text="c)", font=dict(size=16), x=-0.5, y=8, showarrow=False, row=1, col=3)


# Temperature ssp spread (d) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = basins["T2M_change_126"], marker_color= cl[2], name = "SSP126", points=False, opacity=0.7, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins["T2M_change_245"], marker_color= cl[0], name = "SSP245", points=False, opacity=0.7, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins["T2M_change_370"], marker_color= cl[1], name = "SSP370", points=False, opacity=0.7, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins["T2M_change_585"], marker_color= cs[5], name = "SSP585", points=False, opacity=0.7, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins_m["T2M_change_126"],  marker_color=cl[2], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "SSP126",  showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins_m["T2M_change_245"],  marker_color=cl[0], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "SSP245",  showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins_m["T2M_change_370"],  marker_color=cl[1], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "SSP370",  showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins_m["T2M_change_585"],  marker_color=cs[5], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "SSP585",  showlegend = False), row=2, col=3)
fig.update_traces(pointpos = -0.7, box_visible=True, meanline_visible=True, width=1, row = 2, col = 3)

fig.update_yaxes(title = "Temperature change (ºC)", title_standoff = 5, range = [0.8,3.4], dtick = 0.5, side = "right", row = 2, col = 3)
fig.add_annotation(text="d)", font=dict(size=16), x=-0.5, y=3.25, showarrow=False, row=2, col=3)
fig.add_annotation(text="Main catchments", font=dict(size=14), ax=-20, x=0.6, y=1.9, showarrow=True, row=2, col=3)

# layout c and d
fig.update_layout(barmode = "overlay",  plot_bgcolor="rgba(213,213,213,0.6)")
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.5)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.5)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)

# general 
fig.update_layout(autosize = False, width = 900, height = 600, margin = dict(l=10, r=10, b=10, t=30, pad=0, autoexpand=True))
fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_6_GCM_SSP.png", scale=4)
fig.show()